<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Baixando pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make -j4 

In [1]:
#@title Importando bibliotecas {display-mode: "form"}

from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyConformer import *
from pyRAND import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *
from pyMC import *
from pyFullSearch import *
from pyDocker import *

In [2]:
#@title Baixando base de dados proteína ligante {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/pyLiBELa/SB/'

with open(path+'list.txt') as f:
    pdb_codes_list = f.readlines()

pdb_codes_list = [pdb_code[0:4] for pdb_code in pdb_codes_list]

In [3]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.x_dim = 5.0 #@param {type:"number"}
Input.y_dim = 5.0 #@param {type:"number"}
Input.z_dim = 5.0 #@param {type:"number"}
Input.write_grids = True;

In [5]:
import os
import timeit
import numpy as np

num_pairs = 3
num_metrics = 4

dim_grid_list = [5,5,5]

metrics_matrix=np.zeros((num_pairs,num_metrics+1))


for j in range(num_pairs):

  pdb_code = pdb_codes_list[j]
  dim_grid = dim_grid_list[j]

  print(pdb_code,dim_grid)

  Input.x_dim, Input.y_dim, Input.z_dim = dim_grid, dim_grid, dim_grid


  metrics_list = np.zeros(num_metrics+1)

  metrics_list[0] = j
  
  metrics_list[1] = dim_grid

  Input = PARSER()
  Input.grid_prefix = "McGrid_"+pdb_code

  Writer = WRITER(Input)
  Coord = COORD_MC()
  HB = FindHB()

  lig_src =  path + pdb_code +'/' + pdb_code + '.lig.am1bcc.mol2.gz'
  rec_src = path + pdb_code +'/' + pdb_code + '.rec.clean.mol2.gz'

  lig = Mol2(Input, lig_src)
  rec = Mol2(Input, rec_src)
  ref_lig = Mol2(Input, lig_src)

  Energy = Energy2(Input)

  for k in range(len(list(rec.residue_pointer))-1):
    HB.parse_residue(rec.residue_pointer[k]-1, rec.residue_pointer[k+1]-2, rec.resnames[k], rec, lig, 9.0)

  HB.find_ligandHB(lig_src, lig)

  com = Coord.compute_com(lig)

  energy1 = Energy.compute_ene(rec, lig, lig.xyz)
  print("Original energy: %7.3f" % energy1)
  metrics_list[2] = energy1

  Input.write_grids = True

  start_time = timeit.default_timer()
  Grids = Grid(Input, Writer, rec, com)
  time = timeit.default_timer() - start_time
  print('Timeit: %.2f' %time)
  metrics_list[3] = time

  energy2 = Energy.compute_ene(Grids, lig, lig.xyz)
  print("Grid energy: %7.3f" % energy2)
  metrics_list[4] = energy2


  !ls *.grid

  folder = path + pdb_code
  !mv *.grid McLiBELa.log $folder

  print(metrics_list)
  print('')

  metrics_matrix[j] = metrics_list

121P 5
Original energy: -237.910
Timeit: 26.58
Grid energy: 2999795.409
McGrid_121P.grid
[ 0.00000000e+00  5.00000000e+00 -2.37910282e+02  2.65792063e+01
  2.99979541e+06]

181L 5
Original energy: -12.322
Timeit: 24.98
Grid energy: -12.179
McGrid_181L.grid
[  1.           5.         -12.32202051  24.98435135 -12.17868326]

182L 5
Original energy: -17.820
Timeit: 27.82
Grid energy: -17.608
McGrid_182L.grid
[  2.           5.         -17.81983535  27.82236368 -17.6084697 ]



In [6]:
#conferindo nas pastas
for pdb_code in pdb_codes_list[:num_pairs]:
  folder = path+pdb_code+'/'
  !ls $folder

121P.lig.am1bcc.mol2.gz  121P.rec.clean.mol2.gz  McGrid_121P.grid  McLiBELa.log
181L.lig.am1bcc.mol2.gz  181L.rec.clean.mol2.gz  McGrid_181L.grid  McLiBELa.log
182L.lig.am1bcc.mol2.gz  182L.rec.clean.mol2.gz  McGrid_182L.grid  McLiBELa.log


In [10]:
print("{:<10} {:<10} {:<17} {:<10} {:<10}".format('PDB_code','Grid size','Original Energy','Time','Grid Energy'))
for i in range(num_pairs):
  data_line = metrics_matrix[i]
  pdb_code_print = pdb_codes_list[int(data_line[0])]

 #line = pdb_codes_list[int(data_line[0])]+'\t%7.3f\t\t%.2f\t%7.3f'%(data_line[1],data_line[2],data_line[3])
  print("{:<10} {:<10} {:<17} {:<10} {:<10}".format(pdb_code_print,data_line[1],'%7.3f'%data_line[2],'%.2f'%data_line[3], '%7.3f'%data_line[4]) )

PDB_code   Grid size  Original Energy   Time       Grid Energy
121P       5.0        -237.910          26.58      2999795.409
181L       5.0        -12.322           24.98      -12.179   
182L       5.0        -17.820           27.82      -17.608   
